In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [3]:
image_size = [224,224]
data_path = 'Detection'

In [4]:
vgg = VGG16(input_shape= image_size+[3],weights='imagenet',include_top=False)

In [5]:
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [6]:
x = vgg.output
x = GlobalAveragePooling2D()(x)

In [7]:
x = Dense(1024,activation='relu')(x)
x = Dense(1024,activation='relu')(x)
x = Dense(512, activation='relu')(x)

In [8]:
preds = Dense(2,activation='softmax')(x)

In [9]:
model = Model(inputs = vgg.input,outputs=preds)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
for layer in vgg.layers:
    layer.trainable = False

In [12]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('C:/Users/ANANNYA/Desktop/Data/', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 3264 images belonging to 2 classes.


In [13]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
print(train_generator.n)
print(train_generator.batch_size)
print(3264//32)

3264
32
102


In [15]:
step_size_train=train_generator.n//train_generator.batch_size
r = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5)

C:\Users\ANANNYA\AppData\Local\Temp\ipykernel_12772\1856219906.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(generator=train_generator,


Epoch 1/5
102/102 [==============================] - 720s 7s/step - loss: 0.6118 - accuracy: 0.8572
Epoch 2/5
102/102 [==============================] - 726s 7s/step - loss: 0.2427 - accuracy: 0.8873
Epoch 3/5
102/102 [==============================] - 693s 7s/step - loss: 0.2194 - accuracy: 0.8925
Epoch 4/5
102/102 [==============================] - 686s 7s/step - loss: 0.1991 - accuracy: 0.8983
Epoch 5/5
102/102 [==============================] - 648s 6s/step - loss: 0.2029 - accuracy: 0.8961


In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
# plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('brain_tumor.h5')

In [ ]:
img_path = 'C:/Users/ANANNYA/Downloads/Data/Training/no_tumor/image(10).jpg'

In [ ]:
img = image.load_img(img_path,target_size=(224,224))
x= image.img_to_array(img)
x = np.expand_dims(x,axis=0)
img_data = preprocess_input(x)

In [ ]:
rslt = model.predict(img_data)
print(rslt)

In [ ]:
rslt[0][0]

In [ ]:
rslt[0][1]

In [ ]:
if rslt[0][0] == 1:
    prediction = 'Not A Brain Tumor'
else:
    prediction = 'Brain Tumor'
    
print(prediction)